In [6]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from tqdm import tqdm
from copy import deepcopy
import random
import string

from photutils.aperture import CircularAperture, aperture_photometry

from astropy.io import fits
from astropy.wcs import WCS
from astropy.modeling.functional_models import Gaussian2D
from astropy.coordinates import SkyCoord, Angle

from scipy.signal import fftconvolve
from scipy.stats import multivariate_normal

import mastcasjobs

%matplotlib widget

In [2]:
folder = f'/Users/zgl12/Python_Scripts/SynDiff/SkyCells/rings.v3.skycell.2246.020.stk.'

filters = ['r', 'i', 'z', 'y']

file = folder + 'r' + '.unconv.mask.fits'

In [7]:
def catmag_to_imscale(flux,header):
    a = 2.5/np.log(10)
    tmp = (flux - header['boffset']) / (header['bsoften']*2)
    tmp = np.arcsinh(tmp)*a
    return tmp

def ps1_casjobs(ra, dec, radius, maglim=20, quick = True):
    
    name = ''.join(random.choices(string.ascii_letters, k=8))
    
    query = f"""
            SELECT o.objID,
            o.raMean, o.decMean,
            o.qualityFlag,
            o.gMeanPSFMag, o.gMeanPSFMagErr, o.gMeanPSFMagNpt,
            o.rMeanPSFMag, o.rMeanPSFMagErr, o.rMeanPSFMagNpt,
            o.iMeanPSFMag, o.iMeanPSFMagErr, o.iMeanPSFMagNpt,
            o.zMeanPSFMag, o.zMeanPSFMagErr, o.zMeanPSFMagNpt,
            o.yMeanPSFMag, o.yMeanPSFMagErr, o.yMeanPSFMagNpt,
            o.rMeanKronMag, o.rMeanKronMagErr,
            o.iMeanKronMag, o.iMeanKronMagErr,
            o.nDetections,
            o.gFlags, o.gQfPerfect,
            o.rFlags, o.rQfPerfect,
            o.iFlags, o.iQfPerfect,
            o.zFlags, o.zQfPerfect,
            o.yFlags, o.yQfPerfect
            INTO mydb.[{name}]
            from fGetNearbyObjEq({ra}, {dec}, {radius}*60) as x
            JOIN MeanObjectView o on o.ObjID=x.ObjId
            LEFT JOIN StackObjectAttributes AS soa ON soa.objID = x.objID
            WHERE o.nDetections>5
            AND soa.primaryDetection>0
            AND o.iMeanPSFMag < 20
            """
    
    user = "syndiff"
    pwd = "PS1SynDiff"

    jobs = mastcasjobs.MastCasJobs(username=user, password=pwd, context="PanSTARRS_DR2")
    
    if quick:
        table = quick_query(jobs, query)
        if table is None:
            print('Error in casjobs quick query, doing solid query')
            table = solid_query(jobs, query, name)
    else:
        table = solid_query(jobs, query, name)
        
    return table

def quick_query(jobs, query):
    try:
        results = jobs.quick(query, task_name="python cone search")
        return results
        # return results.to_pandas()
    except:
        return None

def solid_query(jobs, query, name):

    results = jobs.submit(query, task_name="python cone search")
    job_status = jobs.status(results)
    
    stsus = False
    print("Finding...", end="", flush=True)
    while stsus == False:
        time.sleep(5)
        print(".", end="", flush=True)
        job_status = jobs.status(results)
        if job_status[1].lower() == 'finished':
            stsus = True
        elif job_status[1].lower() == 'failed':
            print('Error in casjobs query')
            stsus = None
        else:
            stsus = False
            
    if stsus is True:
        
        print("Query finished successfully. Retrieving table...")

        table = jobs.get_table(f"mydb.{name}")
        return table.to_pandas()
    else:
        print('Error in casjobs query')
        


In [4]:
hdul = fits.open(file)
wcs = WCS(hdul[0].header)
data = hdul[0].data

height, width = hdul[0].data.shape
header = hdul[0].header
hdul.close()

# plt.figure()
# plt.subplot(projection=wcs)
# plt.imshow(data, origin='lower', cmap='gray', vmin = np.nanpercentile(data, 10), vmax = np.nanpercentile(data, 90))
# plt.show()

center_pixel = (width / 2, height / 2)

ra, dec = wcs.all_pix2world(center_pixel[0], center_pixel[1], 0)

coords = np.array([ra, dec])

ra = coords[0]
dec = coords[1]

print(ra, dec)

this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]


120.69739070164897 44.96264509147862


In [8]:
cat = ps1_casjobs(ra, dec, 1, maglim=20, quick = False)

Finding....................Query finished successfully. Retrieving table...


In [9]:
x,y = wcs.all_world2pix(cat['raMean'].values,cat['decMean'].values,0)

index = np.where((x > 10) & (x < width -10) & (y > 10) & (y < height-10) & (cat['iMeanKronMag'].values > -5))[0]
cat = cat.iloc[index]

x = x[index]
y = y[index]

cat = cat.reset_index(drop=True)

cat

,objID,raMean,decMean,qualityFlag,gMeanPSFMag,gMeanPSFMagErr,gMeanPSFMagNpt,rMeanPSFMag,rMeanPSFMagErr,rMeanPSFMagNpt,...,gFlags,gQfPerfect,rFlags,rQfPerfect,iFlags,iQfPerfect,zFlags,zQfPerfect,yFlags,yQfPerfect
0,161681208924529464,120.892411,44.740698,52,21.528900,0.044063,8,20.225300,0.018189,11,...,115000,0.999157,115000,0.999403,115000,0.999691,115000,0.998975,115000,0.999575
1,161691207273215522,120.727317,44.745739,53,21.058800,0.033353,11,20.240601,0.019054,15,...,16892216,0.999429,16892216,0.999614,16892216,0.999793,16892216,0.999677,115000,0.999025
2,161691208183558932,120.818320,44.748565,52,21.817499,0.086693,3,20.708000,0.018263,15,...,115000,0.996592,115000,0.999788,115000,0.999820,115000,0.999687,115000,0.999614
3,161701203976027038,120.397541,44.755347,52,20.686001,0.029024,11,19.956499,0.016508,16,...,115000,0.999672,115000,0.999742,115000,0.999860,115000,0.999514,115000,0.999683
4,161701204592836468,120.459250,44.754821,52,22.201200,0.125502,2,21.081600,0.046618,10,...,115000,0.998896,115000,0.999148,115000,0.999540,115000,0.999486,115000,0.998743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,162211206699086984,120.669995,45.180235,61,19.685400,0.057658,12,19.311600,0.031749,14,...,16892216,0.999830,16892216,0.999727,16892216,0.999825,16892216,0.999687,16892216,0.998601
1015,162211206798636201,120.679880,45.179599,60,17.603399,0.003410,10,16.573200,0.004523,11,...,115000,0.999697,115000,0.999758,115000,0.999623,115000,0.999637,115000,0.999132
1016,162211207021062967,120.702136,45.176914,53,19.562599,0.017590,13,19.572201,0.021189,14,...,16892216,0.999775,16892216,0.999681,16892216,0.999701,16892216,0.999588,115000,0.999826
1017,162211207930064233,120.793027,45.177976,60,17.266800,0.002598,7,16.778200,0.002785,18,...,115000,0.999355,115000,0.999366,115000,0.999835,115000,0.999394,115000,0.999624


In [10]:
psf_std=2.5
ps1_zp=25
ps1_band = 'i'

pad = 0

x += pad; y += pad
cat['x'] = x.astype(int); cat['y'] = y.astype(int)

value = 2*np.sqrt(2*np.log(2))

x_indices = np.arange(0, width) # Placing x
y_indices = np.arange(0, height) # Placing y
xx, yy = np.meshgrid(x_indices, y_indices)
xy = np.column_stack([xx.flat, yy.flat]) # Stacking x and y

table = None

band_images = []


for band in ['r', 'i', 'z', 'y']:
    
    cat_image = np.zeros_like(data, dtype = float)

    for i in tqdm(range(len(cat)), desc='Stars'):
        mean = [cat['x'].values[i], cat['y'].values[i]]
        
        try:
            mag = cat[f'{band}MeanKronMag'].values[i]
        except:
            mag = cat[f'{band}MeanPSFMag'].values[i]
            
        if mag < -5:
            continue
        
        flux = 10**((ps1_zp - mag)/2.5) # Amplitude
        # print(mag, flux)
        
        covariance = [[psf_std**2, 0], [0, psf_std**2]] # Covariance matrix for the fit
        gaussian_model = multivariate_normal(mean=mean, cov=covariance) # Gaussian model
        
        model_values = flux*gaussian_model.pdf(xy).reshape(xx.shape) # Gaussians probability distribution function
        
        # print(model_values, type(model_values), model_values.shape, np.nanmax(model_values))

        cat_image += model_values # Create data with the stars
        
    band_images.append(cat_image)
        
    positions = np.zeros((len(cat), 2))
    positions[:,0] = cat['x'].values
    positions[:,1] = cat['y'].values

    aperture = CircularAperture(positions, r=5*3)

    nan_mask = ~np.isfinite(cat_image) # Create mask to exclude NaN pixels
    masked_reference = np.ma.masked_array(cat_image, mask=nan_mask) # Apply aperture photometry while ignoring NaNs
    phot_table = aperture_photometry(masked_reference, aperture)
    
    if table is None:
        table = phot_table.to_pandas()
        table[f'{band}_mag'] = -2.5*np.log10(table['aperture_sum'].values) + ps1_zp
    else:
        table[f'{band}_mag'] = -2.5*np.log10(phot_table['aperture_sum'].value) + ps1_zp
    # break
    
    # break

# cat_image[cat['y'].values,cat['x'].values] = 10**((cat[f'{ps1_band}MeanKronMag'].values-ps1_zp)/-2.5)
# g = Gaussian2D(x_stddev=psf_std,y_stddev=psf_std,x_mean=10,y_mean=10)
# yss,xss = np.mgrid[:21,:21]
# psf = g(xss,yss)
# psf /= np.nansum(psf)
# cat_image = fftconvolve(cat_image, psf, mode='same')

# cat_image[cat_image < 1e-9] = 0

# cat_image = catmag_to_imscale(cat_image,header)

Stars: 100%|██████████| 1019/1019 [34:32<00:00,  2.03s/it]
/var/folders/6v/pg_c9mqn3gn45gn_zmvy9w9cvfsx9j/T/ipykernel_52726/2740297542.py:63: RuntimeWarning: divide by zero encountered in log10
  table[f'{band}_mag'] = -2.5*np.log10(table['aperture_sum'].values) + ps1_zp
Stars: 100%|██████████| 1019/1019 [34:51<00:00,  2.05s/it]


In [12]:
table.to_csv('/Users/zgl12/synth_skycell_phot_table.csv')
cat.to_csv('/Users/zgl12/synth_skycell_catalogue.csv')

bands = ['r', 'i', 'z', 'y']

for i in range(len(['r', 'i', 'z', 'y'])):
    data_to_save = deepcopy(band_images[i])
    hdu = fits.PrimaryHDU(data_to_save, header=deepcopy(header))
    hdul = fits.HDUList([hdu])
    hdul[0].header['FILTER'] = bands[i]
    
    name = f'/Users/zgl12/synth_{header["SKYCELL"]}_{bands[i]}.fits'
    hdul.writeto(name, overwrite=True)
    hdul.close()